# 📘 Materials Characterization: Stress-Strain Analysis> Fit constitutive models to stress-strain data⏱️ **20-30 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Engineering**---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/04_gallery/engineering/materials_characterization.ipynb)


In [ ]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

## 🔬 Domain Background**Model:** $\sigma = E \cdot \epsilon$ (Linear elastic)**Applications:** Young's modulus determination, material testing---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import curve_fit


def linear_elastic(strain, E):
    return E * strain

def power_law(strain, K, n):
    return K * strain**n

## Generate Stress-Strain Data

In [ ]:
E_true = 200  # GPa (steel)
strain = np.linspace(0, 0.002, 30)  # 0.2% strain
stress_true = linear_elastic(strain, E_true)
stress = stress_true + np.random.normal(0, 2, len(strain))

plt.plot(strain*100, stress, 'o', label='Experimental')
plt.xlabel('Strain (%)')
plt.ylabel('Stress (GPa)')
plt.legend()
plt.tight_layout()
plt.tight_layout()
plt.show()


## Determine Young's Modulus

In [ ]:
popt, pcov = curve_fit(linear_elastic, strain, stress, p0=[180])
E_fit = popt[0]
E_err = np.sqrt(pcov[0,0])

print(f'Young\'s Modulus: {E_fit:.1f} ± {E_err:.1f} GPa')
print('\nLiterature value for steel: 200 GPa')
print(f'Error: {abs(E_fit - E_true)/E_true * 100:.1f}%')

## Key Insights1. **Linear region gives elastic modulus**2. **Slope = Young's modulus E**3. **Yield point** where linearity breaks down---